In [22]:
import pynucastro as pyna
reaclibrary = pyna.ReacLibLibrary()
all_nuclei = ["p","n", "h2", "h3", "he3", "he4","Li6","Li7","Be7","Li8","B8","Be9"]#,"B10","B11","C11","B12","C12","N12","C13","N13","C14","N14","O14","N15","O15","O16"]
bbn_library = reaclibrary.linking_nuclei(all_nuclei)
bbn_network = pyna.networks.PythonNetwork(libraries=bbn_library)


In [23]:
p__n_string = '''
b0 = -0.62173 
b1 = 0.22211e2
b2 = -0.72798e2
b3 = 0.11571e3
b4 = -0.11763e2
b5 = 0.45521e2
b6 = -3.7973 
b7 = 0.41266 
b8 = -0.026210
b9 = 0.87934e-3
b10 = -0.12016e-4
qpn = 2.8602

@numba.njit() 
def p__n(rate_eval, tf):  
    # p --> n
    z=5.92989658*tf.T9i
    rate=0
    #rate from https://arxiv.org/pdf/astro-ph/0408076.pdf appendix C
    b=[b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10]
    if tf.T9>1.160451812:
      for i in range(11):
         rate+=1/880.2*np.exp(-qpn*z)*b[i]*z**-i 
        
    #Kawano rate
    #rate=1/879.6*(5.252/z - 16.229/z**2 + 18.059/z**3 + 34.181/z**4 + 27.617/z**5)*np.exp(-2.530988*z)
    rate_eval.p__n = rate

'''

In [24]:
n__p_string = '''
a0 = 1
a1 = 0.15735 
a2 = 4.6172
a3 = -0.40520e2 
a4 = 0.13875e3 
a5 = -0.59898e2
a6 = 0.66752e2 
a7 = -0.16705e2 
a8 = 3.8071
a9 = -0.39140 
a10 = 0.023590 
a11 = -0.83696e-4
a12 = -0.42095e-4 
a13 = 0.17675e-5
qnp = 0.33979 

@numba.njit()
def n__p(rate_eval, tf):
    # n --> p
    z=5.92989658*tf.T9i
    #rate from https://arxiv.org/pdf/astro-ph/0408076.pdf appendix C
    rate=0
    a=[a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13]
    for i in range(14):
      rate+=1/880.2*np.exp(-qnp/z)*a[i]*z**-i

    #Kawano rate
    #rate = 1/879.6*(0.565/z - 6.382/z**2 + 11.108/z**3 + 36.492/z**4 + 27.512/z**5)

    rate_eval.n__p = rate

'''

In [25]:
class n__p_Rate(pyna.Rate):
    def __init__(self, reactants=None, products=None,
                 r0=1.0, T0=1.0, nu=0):

        # we'll take the Q value just to be the change in binding energy
        Q = 0
        for n in reactants:
            Q += -n.A * n.nucbind
        for n in products:
            Q += n.A * n.nucbind

        # we set the chapter to custom so the network knows how to deal with it
        self.chapter = "custom"
        self.reverse = None
    
        # call the Rate init to do the remaining initialization
        super().__init__(reactants=reactants, products=products, Q=Q)

        self.r0 = r0
        self.T0 = T0
        self.nu = nu

    def function_string_py(self):
        """return a string containing a python function that computes
        the rate"""
        return n__p_string

    def eval(self, T, rhoY=None):
        return self.r0 * (T / self.T0)**self.nu

In [26]:
n__p = n__p_Rate(reactants=[pyna.Nucleus("n")],
                  products=[pyna.Nucleus("p")],
                  r0=1, T0=1, nu=1)

In [27]:
class p__n_Rate(pyna.Rate):
    def __init__(self, reactants=None, products=None,
                 r0=1.0, T0=1.0, nu=0):

        # we'll take the Q value just to be the change in binding energy
        Q = 0
        for n in reactants:
            Q += -n.A * n.nucbind
        for n in products:
            Q += n.A * n.nucbind

        # we set the chapter to custom so the network knows how to deal with it
        self.chapter = "custom"

        self.reverse = None
    
        # call the Rate init to do the remaining initialization
        super().__init__(reactants=reactants, products=products, Q=Q)

        self.r0 = r0
        self.T0 = T0
        self.nu = nu

    def function_string_py(self):
        """return a string containing a python function that computes
        the rate"""
        return p__n_string

    def eval(self, T, rhoY=None):
        return self.r0 * (T / self.T0)**self.nu

In [28]:
p__n = p__n_Rate(reactants=[pyna.Nucleus("p")],
                  products=[pyna.Nucleus("n")],
                  r0=1, T0=1, nu=1)

In [29]:
p__n.fname='p__n'
n__p.fname='n__p'
#print(p__n.function_string_py())

In [30]:
bbn_network.validate(reaclibrary)

validation: missing Li8 ⟶ Be8 + e⁻ + 𝜈 as alternative to Li8 ⟶ He4 + He4 + e⁻ + 𝜈 (Q = 16.004 MeV).
validation: missing Be9 + p ⟶ B10 + 𝛾 as alternative to Be9 + p ⟶ n + p + He4 + He4 (Q = 6.586 MeV).
validation: missing Be9 + p ⟶ B10 + 𝛾 as alternative to Be9 + p ⟶ H2 + He4 + He4 (Q = 6.586 MeV).
validation: missing Be9 + p ⟶ B10 + 𝛾 as alternative to Be9 + p ⟶ He4 + Li6 (Q = 6.586 MeV).
validation: missing B8 ⟶ Be8 + e⁺ + 𝜈 as alternative to B8 ⟶ He4 + He4 + e⁺ + 𝜈 (Q = 17.98 MeV).


True

In [31]:
class RatePair:
    """the forward and reverse rates for a single reaction sequence.
    Forward rates are those with Q >= 0.

    :var forward: the forward reaction Rate object
    :var reverse: the reverse reaction Rate object

    """

    def __init__(self, forward=None, reverse=None):
        self.forward = forward
        self.reverse = reverse

    def __repr__(self):
        return f"forward: {self.forward} ; reverse: {self.reverse}"

    def __lt__(self, other):
        if self.forward is not None and other.forward is not None:
            return self.forward < other.forward
        if self.forward is None:
            return False
        return True

    def __eq__(self, other):
        return self.forward == other.forward and self.reverse == other.reverse


In [32]:
bothrates=RatePair(p__n,n__p)

In [33]:
bothrates

forward: p ⟶ n + e⁺ + 𝜈 ; reverse: n ⟶ p + e⁻ + 𝜈

In [34]:
bbn_library += pyna.Library(rates=[p__n,n__p])
bbn_library.remove_rate(bbn_library.get_rate('n__p__weak__wc12'))

In [35]:
bbn_network = pyna.networks.PythonNetwork(libraries=bbn_library)

In [36]:
bbn_network.find_duplicate_links()

[[p + p ⟶ H2 + e⁺ + 𝜈, p + p + e⁻ ⟶ H2 + 𝜈]]

In [37]:
bbn_network.write_network("bbn2_test_integrate.py")

In [38]:
bbn_network.validate(reaclibrary)

validation: missing Li8 ⟶ Be8 + e⁻ + 𝜈 as alternative to Li8 ⟶ He4 + He4 + e⁻ + 𝜈 (Q = 16.004 MeV).
validation: missing Be9 + p ⟶ B10 + 𝛾 as alternative to Be9 + p ⟶ n + p + He4 + He4 (Q = 6.586 MeV).
validation: missing Be9 + p ⟶ B10 + 𝛾 as alternative to Be9 + p ⟶ H2 + He4 + He4 (Q = 6.586 MeV).
validation: missing Be9 + p ⟶ B10 + 𝛾 as alternative to Be9 + p ⟶ He4 + Li6 (Q = 6.586 MeV).
validation: missing B8 ⟶ Be8 + e⁺ + 𝜈 as alternative to B8 ⟶ He4 + He4 + e⁺ + 𝜈 (Q = 17.98 MeV).


True

In [39]:
print(bbn_network.network_overview())

n
  consumed by:
     n + p ⟶ H2 + 𝛾
     H2 + n ⟶ H3 + 𝛾
     He3 + n ⟶ He4 + 𝛾
     Li6 + n ⟶ Li7 + 𝛾
     Li7 + n ⟶ Li8 + 𝛾
     He3 + n ⟶ p + H3
     He3 + n ⟶ H2 + H2
     n + He4 ⟶ H2 + H3
     Li6 + n ⟶ He4 + H3
     Be7 + n ⟶ p + Li7
     Be7 + n ⟶ H2 + Li6
     Be7 + n ⟶ He4 + He4
     Be9 + n ⟶ H2 + Li8
     Be9 + n ⟶ H3 + Li7
     B8 + n ⟶ p + He4 + He4
     n + p + He4 ⟶ Li6 + 𝛾
     n + He4 + He4 ⟶ Be9 + 𝛾
     n + p + p ⟶ p + H2
     n + n + He4 ⟶ H3 + H3
     n + p + He4 ⟶ H3 + He3
     n + He4 + He4 ⟶ p + Li8
     n + He4 + He4 ⟶ H2 + Li7
     n + n + He4 + He4 ⟶ H3 + Li7
     n + p + He4 + He4 ⟶ He3 + Li7
     n + p + He4 + He4 ⟶ H3 + Be7
     n + p + He4 + He4 ⟶ p + Be9
     n ⟶ p + e⁻ + 𝜈
  produced by:
     H2 ⟶ n + p
     H3 ⟶ n + H2
     He4 ⟶ n + He3
     Li7 ⟶ n + Li6
     Li8 ⟶ n + Li7
     Li6 ⟶ n + p + He4
     Be9 ⟶ n + He4 + He4
     H2 + H2 ⟶ n + He3
     H3 + p ⟶ n + He3
     H3 + H2 ⟶ n + He4
     H3 + He4 ⟶ n + Li6
     He4 + He4 ⟶ n + Be7
     Li6 + H2

In [40]:
filter = pyna.RateFilter(products=["He4"], exact=False)
He4rates = bbn_library.filter(filter).get_rates()
He4rates

[Li6 ⟶ He4 + H2,
 Li7 ⟶ He4 + H3,
 Li8 ⟶ He4 + He4 + e⁻ + 𝜈,
 Be7 ⟶ He4 + He3,
 B8 ⟶ He4 + He4 + e⁺ + 𝜈,
 Li6 ⟶ n + p + He4,
 Be9 ⟶ n + He4 + He4,
 H2 + H2 ⟶ He4 + 𝛾,
 H3 + p ⟶ He4 + 𝛾,
 He3 + n ⟶ He4 + 𝛾,
 He3 + p ⟶ He4 + e⁺ + 𝜈,
 H3 + H2 ⟶ n + He4,
 He3 + H2 ⟶ p + He4,
 He3 + H3 ⟶ H2 + He4,
 Li6 + n ⟶ He4 + H3,
 Li6 + p ⟶ He4 + He3,
 Li7 + p ⟶ He4 + He4,
 Be7 + n ⟶ He4 + He4,
 Be9 + p ⟶ He4 + Li6,
 H3 + H3 ⟶ n + n + He4,
 He3 + H3 ⟶ n + p + He4,
 He3 + He3 ⟶ p + p + He4,
 Li7 + H2 ⟶ n + He4 + He4,
 Li8 + p ⟶ n + He4 + He4,
 Be7 + H2 ⟶ p + He4 + He4,
 Be9 + p ⟶ H2 + He4 + He4,
 B8 + n ⟶ p + He4 + He4,
 Li7 + H3 ⟶ n + n + He4 + He4,
 Li7 + He3 ⟶ n + p + He4 + He4,
 Be7 + H3 ⟶ n + p + He4 + He4,
 Be7 + He3 ⟶ p + p + He4 + He4,
 Be9 + p ⟶ n + p + He4 + He4]

In [42]:
TMeV2T9=11.60451812
Tpeak=0.5256676288178614*TMeV2T9

for rate in He4rates:
    print(rate.fname+'\t'+str(rate.eval(Tpeak)))


li6__he4_d	0.0
li7__he4_t	0.0
li8__he4_he4__weak__wc12	0.8251748299560883
be7__he4_he3	0.0
b8__he4_he4__weak__wc12	0.9001912844167906
li6__n_p_he4	0.0
be9__n_he4_he4	0.0
d_d__he4	0.0
p_t__he4	0.0
n_he3__he4	248.91468501748517
p_he3__he4__weak__bet_pos_	0.0
d_t__n_he4	0.0
d_he3__p_he4	0.0
t_he3__d_he4	0.0
n_li6__he4_t	0.0
p_li6__he4_he3	0.0
p_li7__he4_he4	0.0
n_be7__he4_he4	0.4633554890886751
p_be9__he4_li6	0.0
t_t__n_n_he4	0.0
t_he3__n_p_he4	0.0
he3_he3__p_p_he4	0.0
d_li7__n_he4_he4	0.0
p_li8__n_he4_he4	0.0
d_be7__p_he4_he4	0.0
p_be9__d_he4_he4	0.0
n_b8__p_he4_he4	401940012.94586456
t_li7__n_n_he4_he4	0.0
he3_li7__n_p_he4_he4	0.0
t_be7__n_p_he4_he4	0.0
he3_be7__p_p_he4_he4	0.0
p_be9__n_p_he4_he4	0.0


In [43]:
rate

sorted(He4rates,key=lambda rate: rate.eval(Tpeak),reverse=True)

[B8 + n ⟶ p + He4 + He4,
 He3 + n ⟶ He4 + 𝛾,
 B8 ⟶ He4 + He4 + e⁺ + 𝜈,
 Li8 ⟶ He4 + He4 + e⁻ + 𝜈,
 Be7 + n ⟶ He4 + He4,
 Li6 ⟶ He4 + H2,
 Li7 ⟶ He4 + H3,
 Be7 ⟶ He4 + He3,
 Li6 ⟶ n + p + He4,
 Be9 ⟶ n + He4 + He4,
 H2 + H2 ⟶ He4 + 𝛾,
 H3 + p ⟶ He4 + 𝛾,
 He3 + p ⟶ He4 + e⁺ + 𝜈,
 H3 + H2 ⟶ n + He4,
 He3 + H2 ⟶ p + He4,
 He3 + H3 ⟶ H2 + He4,
 Li6 + n ⟶ He4 + H3,
 Li6 + p ⟶ He4 + He3,
 Li7 + p ⟶ He4 + He4,
 Be9 + p ⟶ He4 + Li6,
 H3 + H3 ⟶ n + n + He4,
 He3 + H3 ⟶ n + p + He4,
 He3 + He3 ⟶ p + p + He4,
 Li7 + H2 ⟶ n + He4 + He4,
 Li8 + p ⟶ n + He4 + He4,
 Be7 + H2 ⟶ p + He4 + He4,
 Be9 + p ⟶ H2 + He4 + He4,
 Li7 + H3 ⟶ n + n + He4 + He4,
 Li7 + He3 ⟶ n + p + He4 + He4,
 Be7 + H3 ⟶ n + p + He4 + He4,
 Be7 + He3 ⟶ p + p + He4 + He4,
 Be9 + p ⟶ n + p + He4 + He4]